In [14]:
import requests 
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import datetime
import os

In [5]:
# kospi_list = ["005380","035420","068270","005930","000270","373220","207940","000660","006400","051910"]
df=pd.DataFrame()

kospi_list = {"005380":'현대차',"068270":'셀트리온',"005930":'삼성전자',"000270":'기아',"373220":'LG에너지솔루션',"207940":'삼성바이오로직스',"000660":'sk하이닉스',"006400":'삼성SDI',"051910":"LG화학","015760":"한국전력"}
for code in kospi_list:
    temp_page = 1000
    end_url = 'https://finance.naver.com/research/company_list.naver?keyword=&brokerCode=&writeFromDate=&writeToDate=&searchType=itemCode&itemName='+kospi_list[code]+'&itemCode='+ code + '&page=1000'
    response = requests.get(end_url)
    soup = BeautifulSoup(response.text,"html.parser")
    end_page = soup.select_one('td.on a').text

    for i in range(1,int(end_page)+1):
        page = i
        page_url = 'https://finance.naver.com/research/company_list.naver?keyword=&brokerCode=&writeFromDate=&writeToDate=&searchType=itemCode&itemName='+kospi_list[code]+'&itemCode='+ code + '&page='+str(page)
        page_response = requests.get(page_url)
        page_soup = BeautifulSoup(page_response.text,"html.parser")

        for i in range(1,len(soup.select('a[href*="company_read"]'))):
            try:
                temp = page_soup.select('a[href*="company_read"]')[i]
                parm = temp['href']
                main_url = 'https://finance.naver.com/research/'+parm
                main_response = requests.get(main_url)
                main_soup = BeautifulSoup(main_response.text,"html.parser")


                result=main_soup.select_one('.view_sbj').text.split('\n')

                company=kospi_list[code]
                title = result[2].strip()
                firm = result[3].strip().split('|')[0]
                date = result[3].strip().split('|')[1]
                price = main_soup.select_one('.money').text
                comment = main_soup.select_one('.coment').text
                pdf_link = main_soup.select_one('.view_report a')['href']

                new_data = pd.DataFrame({
                    'code' : [code],
                    'date' : [date],
                    'title' : [title],
                    'firm' : [firm],
                    'coment' : [comment],
                    'price' : [price],
                    'pdf_link' : [pdf_link]

                })
                df = pd.concat([df, new_data])
            except:
                print('오류링크:',main_url)


오류링크: https://finance.naver.com/research/company_read.naver?nid=32843&page=8&searchType=itemCode&itemCode=000660


In [7]:
# df.to_csv('./../../Data/kospi_report.csv',encoding='utf-8-sig')

In [8]:
df['date'] = df['date'].apply(lambda x: pd.to_datetime(x))

In [10]:
# pdf 다운로드 함수
def download(download_url, file_name):
    with open(file_name, "wb") as file:  
        response = requests.get(download_url)
        file.write(response.content)

In [11]:
# 다운로드 경로에 폴더없으면 생성
if not os.path.isdir('./../../Data/report_pdf'):
    os.mkdir('./../../Data/report_pdf')
os.chdir('./../../Data/report_pdf')

In [15]:
# 필요한 기간 인덱스
idx = np.where((df['date']>datetime.datetime(2017, 12, 1))&(df['date']<datetime.datetime(2022, 11, 30)))

In [34]:
df1 = df.iloc[idx]

In [96]:
for i in range(len(df1)):
    url = df1['pdf_link'].iloc[i]
    firm = df1['firm'].iloc[i]
    title = df1['code'].iloc[i] + "_" +firm + '_' + ''.join(df1['date'].iloc[i].strftime('%Y-%m-%d').split('-'))+'.pdf'
    
    print(url, title)
    download(url, title)

https://ssl.pstatic.net/imgstock/upload/research/company/1669245634403.pdf 005380_하이투자증권_20221124.pdf
https://ssl.pstatic.net/imgstock/upload/research/company/1669159096492.pdf 005380_하나증권_20221123.pdf
https://ssl.pstatic.net/imgstock/upload/research/company/1668991395946.pdf 005380_대신증권_20221121.pdf
https://ssl.pstatic.net/imgstock/upload/research/company/1668989536565.pdf 005380_대신증권_20221121.pdf
https://ssl.pstatic.net/imgstock/upload/research/company/1668987378084.pdf 005380_하나증권_20221121.pdf
https://ssl.pstatic.net/imgstock/upload/research/company/1668731440195.pdf 005380_대신증권_20221118.pdf
https://ssl.pstatic.net/imgstock/upload/research/company/1668556182807.pdf 005380_대신증권_20221116.pdf
https://ssl.pstatic.net/imgstock/upload/research/company/1666660931371.pdf 005380_신한투자증권_20221025.pdf
https://ssl.pstatic.net/imgstock/upload/research/company/1666659830672.pdf 005380_유안타증권_20221025.pdf
https://ssl.pstatic.net/imgstock/upload/research/company/1666658751194.pdf 005380_미래에셋증권_202210

In [103]:
df1.loc[df1['code']=='005380'].sort_values('date')

,code,date,title,firm,coment,price,pdf_link
0,005380,2018-01-05,환율 하락이 부정적으로 반영,하나증권,Buy,"170,000",http://imgstock.naver.com/upload/research/company/1515120243660.pdf
0,005380,2018-01-10,"4Q 실적 저조 예상, 환율부담 가중",현대차증권,Buy,"200,000",http://imgstock.naver.com/upload/research/company/1515549021892.pdf
0,005380,2018-01-15,또 한번의 실망감과 점진적 우상향,미래에셋증권,Buy,"170,000",http://imgstock.naver.com/upload/research/company/1515988550331.pdf
0,005380,2018-01-16,"길어진 부진, 1Q18부터는 개선",이베스트증권,Buy,"200,000",http://imgstock.naver.com/upload/research/company/1516068945220.pdf
0,005380,2018-05-18,"가격-비용 개선 효과, 점진 가시화",현대차증권,Buy,"185,000",https://ssl.pstatic.net/imgstock/upload/research/company/1526607164790.pdf
...,...,...,...,...,...,...,...
0,005380,2022-11-21,LA 딜러숍 방문기,하나증권,Buy,"225,000",https://ssl.pstatic.net/imgstock/upload/research/company/1668987378084.pdf
0,005380,2022-11-21,미국 탐방노트 시리즈 #4 HMA/HCA,대신증권,Buy,"220,000",https://ssl.pstatic.net/imgstock/upload/research/company/1668989536565.pdf
0,005380,2022-11-21,미국 탐방노트 시리즈 #3 LA오토쇼/딜러샵,대신증권,Buy,"220,000",https://ssl.pstatic.net/imgstock/upload/research/company/1668991395946.pdf
0,005380,2022-11-23,현대차 북미 CEO와의 만남,하나증권,Buy,"225,000",https://ssl.pstatic.net/imgstock/upload/research/company/1669159096492.pdf
